In [18]:
from openai import OpenAI
import os
import json
from pydantic import BaseModel,Field
from langchain.llms import OpenAI
from dotenv import load_dotenv
import random
from tqdm import tqdm
import re
import pandas as pd
import langsmith as ls
import textwrap
from typing import TypedDict, Optional
from openai import OpenAI
from pydantic import BaseModel
import traceback
from langgraph.graph import StateGraph, END
from typing import TypedDict, Optional
from concurrent.futures import ThreadPoolExecutor, as_completed
from langsmith import traceable, trace
from langsmith import Client, traceable, evaluate
import sys
import os
# Thêm đường dẫn project_root vào sys.path
sys.path.append(os.path.abspath(os.path.join(os.path.dirname("dataset_to_langsmith.ipynb"), '..')))
from In_Context_Learning.preprocess_data import prepare_qa_input_with_answer_filter,standardize_item

In [11]:
file_path = 'test.jsonl'

with open(file_path, "r", encoding="utf-8") as f:
    gsm8k = [json.loads(line) for line in f]

In [22]:
folder_path = ""
filename="tatqa_filtered.json"
file_path = os.path.join(folder_path, filename)
with open(file_path, "r", encoding="utf-8") as f:
    raw_data_tatqa = json.load(f)
tatqa_pre = prepare_qa_input_with_answer_filter(raw_data_tatqa)

In [24]:
folder_path = ""
filename="tabmwp_filtered.json"
file_path = os.path.join(folder_path, filename)
with open(file_path, "r", encoding="utf-8") as f:
    tabmwp_pre = json.load(f)

In [28]:
tatqa = []
for item in tatqa_pre:
    tatqa.extend(standardize_item(item, "tatqa"))
tatqa

[{'question': 'How much is the 2019 rate of inflation?',
  'answer': '2.9',
  'context': 'Actuarial assumptions\n\nThe Group’s scheme liabilities are measured using the projected unit credit method using the principal actuarial assumptions set out below:\n\nNotes: 1 Figures shown represent a weighted average assumption of the individual schemes.\n\n2 The rate of increases in pensions in payment and deferred revaluation are dependent on the rate of inflation.\n\n|  | 2019 % | 2018 % | 2017 % |\n| --- | --- | --- | --- |\n| Weighted average actuarial assumptions used at 31 March1: |  |  |  |\n| Rate of inflation2 | 2.9 | 2.9 | 3.0 |\n| Rate of increase in salaries | 2.7 | 2.7 | 2.6 |\n| Discount rate | 2.3 | 2.5 | 2.6 |\n'},
 {'question': 'How much is the 2018 rate of inflation?',
  'answer': '2.9',
  'context': 'Actuarial assumptions\n\nThe Group’s scheme liabilities are measured using the projected unit credit method using the principal actuarial assumptions set out below:\n\nNotes: 1 

In [27]:
tabmwp = []
for item in tabmwp_pre:
    tabmwp.extend(standardize_item(item, "tabmwp"))
tabmwp

[{'question': "The members of the local garden club tallied the number of plants in each person's garden. How many gardens have at least 47 plants?",
  'answer': '13',
  'context': 'Plants per garden\n\n| Stem | Leaf  |\n| --- | --- |\n| 3 | 3, 3, 3, 5, 5 |\n| 4 | 6 |\n| 5 | 4, 5, 7, 8 |\n| 6 | 7, 8 |\n| 7 | 2, 3, 7, 9 |\n| 8 | 6, 8, 9 |\n'},
 {'question': 'The Burlington Symphony categorizes its donors as gold, silver, or bronze depending on the amount donated. What fraction of donors are at the bronze level? Simplify your answer.',
  'answer': '58/141',
  'context': 'Donor levels\n\n| Donation level | Number of donors |\n| --- | --- |\n| Gold | 15 |\n| Silver | 68 |\n| Bronze | 58 |\n'},
 {'question': "Employees at Eve's Movies tracked the number of movies that customers rented last month. How many customers are there in all?",
  'answer': '44',
  'context': 'Renting movies last month\n\n| Number of times | Frequency |\n| --- | --- |\n| 0 | 1 |\n| 1 | 18 |\n| 2 | 12 |\n| 3 | 13 |\n| 

In [8]:
load_dotenv()

True

In [30]:
client = Client()

In [ ]:
sample_size = 300
random.seed(42)  
test_samples = random.sample(list(tatqa), sample_size)

dataset_id = "04f51027-12f7-4ffc-b682-25bf1d978fcb"

inputs = [{"question": item["question"], "context": item["context"]} for item in test_samples]
outputs = [{"answer": item["answer"]} for item in test_samples]

client.create_examples(
    inputs=inputs,
    outputs=outputs,
    dataset_id=dataset_id,
)


{'example_ids': ['b6e86b66-aef0-4abe-a5bc-88faf94978e3',
  '47473aa0-d51d-4bbb-ad1d-0db347c69899',
  '3b23260c-c869-45e7-9800-81c0df4daf6a',
  'cdc81b17-159a-4500-844b-b74d98754d70',
  '65b79381-99d5-4df1-880f-57b3b3ab706f',
  '8818b552-b695-43b8-8e3e-d7325bc6b9f9',
  'fb80de9e-dcc6-42ff-886a-8aece6cb019b',
  '6394ae57-b3dc-4b30-89b1-a2c2d15dfdab',
  '5141640c-74e0-4937-8764-e56177346603',
  'e29238eb-92a4-41d1-8407-655c9e314fcd',
  '4368d6cd-5407-484a-9835-ca5ff13cf4c9',
  'd3da0324-9ee6-475e-9487-323308ff0138',
  '9e8c036c-c3ba-4117-ac30-422358722ce6',
  'c831b819-6a6a-4d10-9934-7d06841660a6',
  '9cd6b2ad-518b-47c1-abff-54abe3bbe256',
  '7659849e-e9a2-4bcc-9559-8512fe293bf2',
  '844e6888-9906-474b-99bc-51e1e620465b',
  'db1738ff-9246-4363-b2ee-bac6f6644471',
  '47b63b08-246e-40e2-b824-1d23b5cf4d0a',
  '1e92e499-4646-4d22-a250-9308b12e1145',
  '86fe27d8-7440-4ae6-92ea-3fbce1316993',
  '8b87d25f-a785-4444-8939-227d7afd176d',
  'c1c99b7d-d7e5-4be8-b703-9a8642212bd0',
  '1d8acf88-c570-42

In [38]:
sample_size = 300
random.seed(42)  
test_samples = random.sample(list(tabmwp), sample_size)

dataset_id = "d6cf4f6c-2033-4450-94eb-fc984413dfe9"

inputs = [{"question": item["question"], "context": item["context"]} for item in test_samples]
outputs = [{"answer": item["answer"]} for item in test_samples]

client.create_examples(
    inputs=inputs,
    outputs=outputs,
    dataset_id=dataset_id,
)


{'example_ids': ['a1ebeb78-09ac-4d7f-89e3-c30dc34639ce',
  'fe5b67ce-a394-404b-8220-a45c8386f959',
  '944fd734-5b90-4b59-90cf-e2cbdd836634',
  '2208760f-e00c-4768-96b8-63c84db27fff',
  '957a5028-891f-470e-b369-2f4632ddd243',
  'd96337f4-4e96-44b2-9bfc-3800c16ed6ef',
  'b57b0c9d-037c-4d54-a39e-c79ed8a5bd40',
  'd9448c11-fed2-4012-9d83-87d036a2ca6c',
  '79441866-b1fb-4886-a6e0-6c58a100b3b7',
  '53460478-2261-43b5-a27f-6c122c168990',
  'aa22b933-5d80-4f76-9b76-ffebc011ede8',
  '9ace4a08-53e5-481f-81c5-c30ba98803c9',
  '80cf40a6-815d-498e-a03d-f3b0e2bd21f6',
  'b193dcc3-d389-4318-9d87-afb2945e6628',
  '2fd61d4f-e8a8-4126-8c6c-d7c58984e2f0',
  '7afba7ee-9e2e-41c3-aa45-75ea8363655d',
  'ffb7115b-351b-4c65-88cc-4b32901b5d64',
  '0e552686-c372-410b-81d4-3cb897f2d4b4',
  'e25e317a-440e-47cb-852c-8b26a2112c7a',
  'de84105b-522e-4328-b3ea-1f8daccee009',
  '21f2d886-26f3-41fe-b1e7-9b9e224f0045',
  'bc573c0b-deb1-4b11-83d8-1bcf236f3709',
  '399ac6da-0920-4f8c-ad33-51e1af21358e',
  '76951af5-9607-44